In [4]:
import json
import plotly.graph_objects as go

# Load your JSON
with open("sdk_functions.json") as f:
    data = json.load(f)

# Group HACL* functions by category
def classify_hacl_function(name):
    if "MD5" in name:
        return "HACL_MD5"
    elif "SHA2" in name or "sha2" in name:
        return "HACL_SHA2"
    elif "SHA3" in name or "sha3" in name:
        return "HACL_SHA3"
    elif "Blake2" in name:
        return "HACL_Blake2"
    elif "Chacha20" in name or "chacha20" in name:
        return "HACL_Chacha20"
    elif "Poly1305" in name:
        return "HACL_Poly1305"
    else:
        return "HACL_Other"

hacl_funcs = [f for f in data if f["is_hacl"]]
runtime_funcs = [f for f in data if not f["is_hacl"]]

# Top 5 runtime functions by size
runtime_funcs.sort(key=lambda x: x["size_bytes"], reverse=True)
top_runtime = runtime_funcs[:5]
other_runtime = runtime_funcs[5:]
other_runtime_size = sum(f["size_bytes"] for f in other_runtime)

# Setup labels and graph structure
labels = []
label_map = {}

def get_label(label):
    if label not in label_map:
        label_map[label] = len(labels)
        labels.append(label)
    return label_map[label]

sources = []
targets = []
values = []

# Root
root = get_label("Full Binary")

# First level
hacl = get_label("HACL*")
runtime = get_label("Runtime")

sources += [root, root]
targets += [hacl, runtime]
values += [
    sum(f["size_bytes"] for f in hacl_funcs),
    sum(f["size_bytes"] for f in top_runtime) + other_runtime_size
]

# HACL classes
hacl_classes = {}
for f in hacl_funcs:
    cls = classify_hacl_function(f["name"])
    if cls not in hacl_classes:
        hacl_classes[cls] = []
    hacl_classes[cls].append(f)

for cls, funcs in hacl_classes.items():
    class_node = get_label(cls)
    sources.append(hacl)
    targets.append(class_node)
    values.append(sum(f["size_bytes"] for f in funcs))
    for f in funcs:
        func_node = get_label(f["name"])
        sources.append(class_node)
        targets.append(func_node)
        values.append(f["size_bytes"])

# Runtime functions
for f in top_runtime:
    tgt = get_label(f["name"])
    sources.append(runtime)
    targets.append(tgt)
    values.append(f["size_bytes"])

if other_runtime_size > 0:
    tgt = get_label("Other Runtime")
    sources.append(runtime)
    targets.append(tgt)
    values.append(other_runtime_size)

# Plot
fig = go.Figure(go.Sankey(
    arrangement="snap",
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
    )
))

fig.update_layout(
    title_text="Binary Breakdown: HACL* Categories vs Runtime",
    font_size=12,
    height=900  # make it taller
)

fig.show()
